In [80]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import time
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [86]:
train_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
    ])
train_set = datasets.ImageFolder(root='data/001/train',transform=train_transform)
train_loader = torch.utils.data.DataLoader(train_set,
                                             batch_size=4, shuffle=True,
                                             num_workers=8)
test_set = datasets.ImageFolder(root='data/001/test',transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_set,
                                             batch_size=4, shuffle=True,
                                             num_workers=8)

In [107]:
def train(model, device, criterion, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        loss = criterion(outputs, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {:2d} [{:5d}/{} ({:2.1f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [108]:
def test(model, device, criterion, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            test_loss += criterion(output, target).item() 
            pred = output.argmax(dim=1)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [109]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available(): print(torch.cuda.get_device_name(0))

In [110]:
model = models.resnet18(pretrained=True).to(device)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(model.fc.in_features, n)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

for epoch in range(2):
    train(model, device, criterion, train_loader, optimizer, epoch)
    test(model, device, criterion, test_loader)

Train Epoch:  0 [    0/4654 (0%)]	Loss: 2.488176
Train Epoch:  0 [   40/4654 (1%)]	Loss: 3.111938
Train Epoch:  0 [   80/4654 (2%)]	Loss: 2.292539
Train Epoch:  0 [  120/4654 (3%)]	Loss: 2.431450
Train Epoch:  0 [  160/4654 (3%)]	Loss: 2.121452
Train Epoch:  0 [  200/4654 (4%)]	Loss: 3.081182
Train Epoch:  0 [  240/4654 (5%)]	Loss: 2.206930
Train Epoch:  0 [  280/4654 (6%)]	Loss: 2.237951
Train Epoch:  0 [  320/4654 (7%)]	Loss: 1.683698
Train Epoch:  0 [  360/4654 (8%)]	Loss: 1.622422
Train Epoch:  0 [  400/4654 (9%)]	Loss: 1.441098
Train Epoch:  0 [  440/4654 (9%)]	Loss: 1.389880
Train Epoch:  0 [  480/4654 (10%)]	Loss: 1.511540
Train Epoch:  0 [  520/4654 (11%)]	Loss: 1.600518
Train Epoch:  0 [  560/4654 (12%)]	Loss: 1.511854
Train Epoch:  0 [  600/4654 (13%)]	Loss: 2.052223
Train Epoch:  0 [  640/4654 (14%)]	Loss: 1.312982
Train Epoch:  0 [  680/4654 (15%)]	Loss: 2.546352
Train Epoch:  0 [  720/4654 (15%)]	Loss: 2.311950
Train Epoch:  0 [  760/4654 (16%)]	Loss: 0.879257
Train Epoch: